# CCD calculations

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## CCD calculations on H2O // STO-3G

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz")
N = molecule.numberOfElectrons()
N_P = N // 2

print(molecule)

Number of electrons: 10
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)



The first task is to generate the RHF reference, so we'll do an RHF SCF calculation.

In [3]:
r_spinor_basis = gqcpy.RSpinorBasis(molecule, "STO-3G")
K = r_spinor_basis.numberOfSpatialOrbitals()

S = r_spinor_basis.quantizeOverlapOperator().parameters()

In [4]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(r_spinor_basis, molecule)  # 'sq' for 'second-quantized'

The current Hamiltionian is expressed in the non-orthogonal AO basis. This is exactly what we need to start an RHF calculation.

In [ ]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, sq_hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective(sq_hamiltonian)  # use the default threshold of 1.0e-08

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so they will represent the canonical RHF spin-orbital basis.

In [ ]:
rhf_qc_structure = gqcpy.RHF.optimize(objective, solver, environment)

In [ ]:
rhf_energy = rhf_qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value()
print(rhf_energy)

This is very much in line with `crawdad`'s results: `-74.942079928192`.

In [ ]:
rhf_parameters = rhf_qc_structure.groundStateParameters()
C = rhf_parameters.coefficientMatrix()

In [ ]:
r_spinor_basis.transform(C)  # now represents a restricted spin-orbital basis using the canonical RHF MOs

Before continuing with the CCSD specifics, we must first prepare the molecular Hamiltonian in the correct spinor basis. Since we have implemented CCSD using general spinors, we should use a `GSpinorBasis`.

In [ ]:
g_spinor_basis = gqcpy.GSpinorBasis.FromRestricted(r_spinor_basis)  # represents a general spinor basis, based off the restricted canonical RHF MOs

M = g_spinor_basis.numberOfSpinors()
ghf_onv = gqcpy.SpinUnresolvedONV.GHF(M, N, rhf_parameters.spinOrbitalEnergiesBlocked())

orbital_space = ghf_onv.orbitalSpace()
print(orbital_space.description())

CCSD is also implemented as a `QCMethod`. This means that we'll have to set up an environment and a particular solver.

In [ ]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(g_spinor_basis, molecule)

In [ ]:
solver = gqcpy.CCDSolver.Plain()
environment = gqcpy.CCSDEnvironment.PerturbativeCCD(sq_hamiltonian, orbital_space)

Let's now proceed by actually optimizing the CCD wave function model. This may take a while if `gqcp` is built in Debug mode!

In [ ]:
ccd_qc_structure = gqcpy.CCD.optimize(solver, environment)

In [ ]:
ccd_correlation_energy = ccd_qc_structure.groundStateEnergy()
print(ccd_correlation_energy)